In [166]:
"""
Скрипт для обработки данных  с портала трудвсем для БЦК
"""

'\nСкрипт для обработки данных  с портала трудвсем для БЦК\n'

In [167]:
import pandas as pd
import numpy as np
import openpyxl
import json
import ast
import re
import os
import datetime
import time
from dateutil import parser
pd.options.mode.chained_assignment = None


In [168]:
def extract_data_from_list_cell(cell:str,lst_need_keys:list):
    """
    Функция для извлечения данных из словаря в ячейке датафрейма
    """
    try:
        
        lst_lang = ast.literal_eval(cell) # превращаем в список
        if lst_lang:            
            out_str_lst = [] # создаем список содержащий выходные строки
            for lang_dict in lst_lang:
                lst_lang_str = [] # список для хранения значений извлеченных из словаря
                for idx,key in enumerate(lst_need_keys):
                    lst_lang_str.append(lang_dict.get(key,None))             
                    
                lst_lang_str = [value for value in lst_lang_str if value] # отбрасываем None
                single_lang_str = ','.join(lst_lang_str) # создаем строку для одного языка
                out_str_lst.append(single_lang_str)
            return ';'.join(out_str_lst)
                               
        else:
            return 'Не указано'
        

    except FileNotFoundError:
        return 'Не удалось обработать содержимое ячейки'

    

In [169]:
def clear_tag(cell):
    """
    Функция для очистки текста от тегов HTML
    """
    value = str(cell)
    if value != 'nan':        
        result = re.sub(r'<.*?>','',value)
        result = re.sub(r'&[a-z]*?;','',result)
        return result
    else:
        return None

def clear_bonus_tag_br(cell):
    """
    Функция для очистки данных в колонке Дополнительные бонусы
    """
    cell = str(cell)

    if cell !='nan':
        value = str(cell)
        result = re.sub(r'<.*?>','.',value)

        return result
    else:
        return None

In [170]:
def convert_date(cell):
    """
    Функция конвертации строки содержащей дату и время
    """
    value = str(cell)
    try:        
        if value !='nan':
            date_time = parser.parse(value).date() # извлекаем дату
            date_time = datetime.datetime.strftime(date_time,'%d.%m.%Y') # конфертируем в нужный формат
            return date_time

        else:
            return None
    except:
        return 'Не удалось обработать содержимое ячейки'
    

In [171]:
def prepare_data_vacancy(df:pd.DataFrame,dct_name_columns:dict,lst_columns:list)->pd.DataFrame:
    """
    Функция для обработки датафрейма с данными работы в России
    """
    df = df[dct_name_columns.keys()]
    df.rename(columns=dct_name_columns,inplace=True)
    # Обрабатываем обычные колонки
    
    df['Дополнительные бонусы'] = df['Дополнительные бонусы'].apply(clear_bonus_tag_br)
    df['Требования'] = df['Требования'].apply(clear_tag)
    df['Обязанности'] = df['Обязанности'].apply(clear_tag)
    
    # Числовые
    lst_number_columns = ['Требуемый опыт работы в годах','Минимальная зарплата','Максимальная зарплата','Количество рабочих мест']
    df[lst_number_columns] = df[lst_number_columns].fillna(0)
    df[lst_number_columns] = df[lst_number_columns].astype(int,errors='ignore')
  
    # Временные
    
    df['Дата размещения вакансии'] = df['Дата размещения вакансии'].apply(convert_date)
    df['Дата изменения вакансии'] = df['Дата изменения вакансии'].apply(convert_date)
    
    # Категориальные
    df['Квотируемое место'] = df['Квотируемое место'].apply(lambda x: 'Квотируемое место' if x == 'true' else None)
    df['Статус проверки вакансии'] = df['Статус проверки вакансии'].replace(dct_status_vacancy)
    # Начинаем извлекать данные из сложных колонок с json
    # данные по работодателю
    df['КПП работодателя'] = df['Данные компании'].apply(lambda x: json.loads(x).get('kpp','Не указано'))
    df['ОГРН работодателя'] = df['Данные компании'].apply(lambda x: json.loads(x).get('ogrn','Не указано'))
    df['Контактный телефон'] = df['Данные компании'].apply(lambda x: json.loads(x).get('phone','Не указано'))
    df['Email работодателя'] = df['Данные компании'].apply(lambda x: json.loads(x).get('email','Не указано'))
    df['Профиль работодателя'] = df['Данные компании'].apply(lambda x: json.loads(x).get('url','Не указано'))
    
    # Обрабатываем колонку с языками
    df['Требуемые языки'] = df['Данные по языкам'].apply(lambda x:extract_data_from_list_cell(x,['code_language','level']))
    df['Требуемые хардскиллы'] = df['Данные по хардскиллам'].apply(lambda x:extract_data_from_list_cell(x,['hard_skill_name']))
    df['Требуемые софтскиллы'] = df['Данные по софтскиллам'].apply(lambda x:extract_data_from_list_cell(x,['soft_skill_name']))
    
    df.drop(columns=['Данные компании','Данные по языкам','Данные по хардскиллам','Данные по софтскиллам'],inplace=True)
    
    df = df.reindex(columns=lst_columns)
    
    return df
    
    

In [172]:
region = 'Приморский край'
file_data = 'data/vacancy.csv'
file_org = 'data/company.xlsx'
end_folder = 'data'

In [173]:
# колонки которые нужно оставить и переименовать
dct_name_columns = {'busy_type':'Тип занятости','contact_person':'Контактное лицо','date_create':'Дата размещения вакансии',
                    'date_modify':'Дата изменения вакансии','education':'Образование','education_speciality':'Требуемая специализация','is_quoted':'Квотируемое место',
                    'need_medcard':'Требуется медкнижка',
                   'other_vacancy_benefit':'Дополнительные бонусы','position_requirements':'Требования',
                    'position_responsibilities':'Обязанности','regionName':'Регион','required_experience':'Требуемый опыт работы в годах',
                    'retraining_capability':'Возможность переподготовки','required_certificates':'Требуемые доп. документы',
                   'required_drive_license':'Требуемые водительские права','retraining_grant':'Стипендия',
                    'retraining_grant_value':'Размер стипендии','salary':'Зарплата','salary_min':'Минимальная зарплата','salary_max':'Максимальная зарплата',
                    'schedule_type':'График работы','social_protected_ids':'Социально защищенная категория',
                   'source_type':'Источник вакансии','status':'Статус проверки вакансии','transport_compensation':'Компенсация транспорт',
                    'vacancy_address_additional_info':'Доп информация по адресу вакансии','vacancy_address':'Адрес вакансии',
                    'vacancy_address_latitude':'Долгота адрес вакансии','vacancy_address_longitude':'Широта адрес вакансии',
                   'vacancy_benefit_ids':'Бонусы','vacancy_name':'Вакансия','work_places':'Количество рабочих мест','professionalSphereName':'Сфера деятельности',
                    'full_company_name':'Полное название работодателя','company_inn':'ИНН работодателя','company':'Данные компании',
                    'languageKnowledge':'Данные по языкам','hardSkills':'Данные по хардскиллам','softSkills':'Данные по софтскиллам'}

In [174]:
dct_status_vacancy = {'ACCEPTED':'Данные вакансии проверены работодателем','AUTOMODERATION':'Автомодерация',
                      'REJECTED':'Отклонено','CHANGED':'Статус вакансии изменен',
                      'WAITING':'Ожидает подтверждения',}

In [175]:
t = time.localtime()  # получаем текущее время
current_time = time.strftime('%H_%M_%S', t)
current_date = time.strftime('%d_%m_%Y',t)

In [176]:
df = pd.read_csv(file_data,encoding='UTF-8',sep='|',dtype=str)


In [177]:
company_df = pd.read_excel(file_org,dtype=str)

In [178]:
company_df

,Краткое название организации,ИНН
0,"АО ""Улан-Удэнский авиационный завод""",0323018510
1,"АО ""У-У ППО""",0323053578
2,"АО ""Улан-Удэнский авиационный завод"" (ООО ""Аэр...",0326568462


In [179]:
lst_columns = ['Дата размещения вакансии','Дата изменения вакансии','Регион','Вакансия','Сфера деятельности','Количество рабочих мест',
               'Зарплата','Минимальная зарплата','Максимальная зарплата','График работы','Тип занятости','Образование','Требуемая специализация',
               'Требования','Обязанности','Бонусы','Дополнительные бонусы','Возможность переподготовки','Стипендия','Размер стипендии','Компенсация транспорт',
               'Квотируемое место','Социально защищенная категория',
               'Требуемый опыт работы в годах','Требуется медкнижка','Требуемые доп. документы','Требуемые водительские права',
               'Требуемые языки','Требуемые хардскиллы','Требуемые софтскиллы',
               'Источник вакансии','Статус проверки вакансии','Полное название работодателя','Адрес вакансии','Доп информация по адресу вакансии',
               'ИНН работодателя','КПП работодателя','ОГРН работодателя','Контактное лицо','Контактный телефон','Email работодателя',
               'Профиль работодателя','Долгота адрес вакансии','Широта адрес вакансии']

In [180]:
df = df[df['regionName'] == region]

In [181]:
prepared_df = prepare_data_vacancy(df,dct_name_columns,lst_columns)
union_company_df_columns = list(prepared_df.columns).insert(0,'Организация')
union_company_df = pd.DataFrame(columns=union_company_df_columns)


In [182]:
dct_df = dict() # словарь для хранения данных для каждого инн
used_name_sheet = set() # множество для хранения использованных названий листов

In [183]:
org_folder = f'{end_folder}/Вакансии по организациям/{current_date}' # создаем папку куда будем складывать вакансии по организациям
if not os.path.exists(org_folder):
    os.makedirs(org_folder)


# Собираем датафреймы по ИНН
for idx,row in enumerate(company_df.itertuples()):
    name_company = row[1] # название компании
    
    inn_company = row[2] # инн компании
    temp_df = prepared_df[prepared_df['ИНН работодателя'] ==inn_company] # фильтруем по инн
    temp_df.sort_values(by=['Вакансия'],inplace=True) # сортируем
    temp_df.to_excel(f'{org_folder}/{name_company}.xlsx',index=False) # сохраняем
    # создаем отдельный файл в котором будут все вакансии по выбранным компаниям
    temp_df.insert(0,'Организация',name_company)
    union_company_df = pd.concat([union_company_df,temp_df],ignore_index=True)
    
    
    

OSError: [Errno 22] Invalid argument: 'data/Вакансии по организациям/14_03_2024/АО "Улан-Удэнский авиационный завод".xlsx'

In [ ]:
union_company_df

In [ ]:
union_company_df.to_excel(f'{org_folder}/Общий файл.xlsx',index=False)

In [ ]:
with pd.ExcelWriter(f'{end_folder}/Данные по региону от {current_time}.xlsx') as writer:
    prepared_df.to_excel(writer,sheet_name='Обработанные данные',index=0)
    df.to_excel(writer,sheet_name='Необработанные данные',index=1)

In [ ]:
print('Lindy Booth!')